# Creating a custom kernel

A few weeks ago we discussed using [Markdown in code cells](2018-07-06-Markdown-code-cells.ipynb).  The implementation injected a function into the input transform manager in a running kernel.  

This notebook takes an alternate approach where we create a custom kernel that permits markdown.  *Most of this work was made possible by the demonstration in [creating a simple kernel for Jupyter](https://github.com/ipython-books/cookbook-2nd/blob/master/chapter01_basic/06_kernel.md)*.

## Reusing the our old `Literate` transformer

The Literate transformer will extract the indented code using CommonMark.

In [9]:
    from deathbeds.__Markdown_code_cells import Literate
    transformer = Literate()

## Defining the Kernel Class

The `Kernel` subclasses the existing `IPythonKernel` and strips the indented code before executing the kernel as normal.

In [10]:
    from ipykernel.ipkernel import IPythonKernel
    class LiterateKernel(IPythonKernel):
        implementation = 'Literate'
        implementation_version = '0.0.1'
        banner = "Literate"

        def do_execute(self, code, silent, store_history=True, user_expressions=None, allow_stdin=False):
            code = transformer(code)
            return super().do_execute(
                code, 
                silent, store_history=store_history, user_expressions=user_expressions, allow_stdin=allow_stdin)

## Defining the `Kernel` assets

In [12]:
    import json; from pathlib import Path

### `kernel.json`

In [47]:
Path('literate/kernel.json').write_text(
    json.dumps({
     "argv": "python -m deathbeds.literate_kernel -f {connection_file}".split(),
     "display_name": "creole",
     "name": "creole",
     "language": "python"
    })
)

> Originally we tried running the new kernel with IPython, but that is not permitted.  `python` must invoke the `kernelspec`.  

### The python script

The python script imports this post and creates the module that will run the kernel.

In [14]:
Path('literate_kernel.py').write_text("""
with __import__('importnb').Notebook():
    from deathbeds.__Literate_Markdown_Kernel import LiterateKernel

if __name__ == '__main__':
    from ipykernel.kernelapp import IPKernelApp
    IPKernelApp.launch_instance(kernel_class=LiterateKernel)""");

In short, this notebook defines our literate kernel and `deathbeds.literate_kernel` runs it.

## Installation

Run these commands in the `deathbeds` module directory.

    jupyter kernelspec install --user literate
    jupyter kernelspec list

## Tests

In [15]:
    def test_import_kernel():
        from deathbeds import literate_kernel